# Deep Learning
> A catalogue of not so deep ideas in Deep Learning.
- toc: true 
- badges: true
- comments: true
- categories: [Research]


Deep Learning is a collection of techniques for finding functions that can approximate arbitrary input-output mappings, while capturing enough structure of the problem to be able to use them in practical applicatons.language-translation systems,Image classification,Movie Recommendation systems are Some notable applications where they have become defacto-choice. This is by no means an exhaustive treatment of the field. Here I provide concise summaries,followed by a simple implementation of some of the most interesting ideas in the field. I believe that many things in the field are unnecessarily complicated by lengthy treatment where a readable code and short explanation will suffice. 

`Note` : This is not intended to be the first introduction to deep learning. Here I wanted to succintly catalogue some the latest advancements in the field. But, respecting tradition the first module starts from the basics. Only hard prerequisite is to have a good intuition of `matrix multiplication` notion of `taking a derivative`.

## Basics

Many deep learning models follow a simple recipe:

    1. Gather the data.
    2. Define learnable parameters. And specify how they will interact with the data.(architecture)
    3. Define a loss function to minimize.
    4. Adjust the parameters until satisfied.

### Train a linear regression model using gradient-descent.

## Deep RL Methods

### Motivation

Deep Learning allows for learning generalizable mappings between input and output. In supervised setting we are given a `fixed dataset` D = {$(x_{i},y_{i})$},and are tasked to predict $y_{i}$ using $x_{i}$. Thus, we know groud truth for all the input data. This let's us formulate a `loss` that reflects our dissatisfaction with the predicted `outputs` and optimize over it. But, consider how we learn to perform any new task ? The dataset and the learning signal comes sequentially and is also dependent on our actions. We don't have prepared datasets in real life. We learn from experience. `RL` deals with learning under this natural setting. The key difference to note is that the dataset is not `constant`, It changes everytime, contingent on your actions and the stochasticity in the environment. To make this clear, consider learning to play a video game. The pixels (call it `state`) and the score you receive ( call it `reward`) cannot be predetermined until you actually go through the experience. Here the dataset D = {$(state_{i},reward_{i})$} is not same everytime you play the game. `RL` provides a formalism for learning optimal decision making. This will become clear when we see some concrete examples and code. But combing these techniques with `Neural networks` has given us general algorithms that `learn to play atari games`, `beat world champions at Go` and `train robots to learn simple tasks`.
![](my_icons/rlsuc.png "Credit: http://rail.eecs.berkeley.edu/deeprlcourse/static/slides/lec-1.pdf")

### Problem

Consider the Following optimization problem. Find the shortest path from state $s_0$ to goal $g$,where the edges indicate the cost/distance ?

![](my_icons/sp.png "Credit: https://nanjiang.cs.illinois.edu/files/cs598/slides_intro_f20.pdf")